# Django in Depth
- Tutorial, updated for Django 1.8
- James Bennet
- 2015-04-09

## The ORM

### Database Backends
There are a set of classes used to support the four built-in backends, plus a base class that can be used to write your own backend for un-supported databases or database drivers.

### `SQLCompiler`
This is the bridge between high-level queries and the database backend.

It's very complex code.

All the magic happens in `as_sql()`.

### `Query`
Data structure and methods representing a database query. It's a tree-like data structure. 

Two flavors: 
- `Query`: normal ORM operations
- `RawQuery`: real, custom SQL queries, that returns something resembling what a normal `Query` would expect

Also scary, complex code. It has to support infinite and arbitrary chaining of ORM methods (`.filter()`, `.distinct()`, etc.). Most of the code comes from this merging process.

You shouldn't often need to subclass `Query`, but rather `QuerySet`.

### Custom Lookups
All your custom stuff gets access to the Django internals, like `as_sql()` and the `SQLCompiler` instance. Some ways to do custom stuff:
- `F objects`
- `Q objects`
- `django.db.models.Lookup`
- `django.db.models.Transform`

### `QuerySet`
Wraps `Query` in a nice API. It's lazy; doesn't touch the database until you force it to, by calling a method that doesn't return a `QuerySet`. 

Printing a `QuerySet`, say, in the Django shell, it'll only show the first 21 objects... `__repr__` adds a `LIMIT 21` to the query, so that you don't accidentally run out of memory trying to build a string with a million results in it.

It's a container - it stores results to a cache that is only every populated once.

All operations except for iteration and length/existence checks perform a new query and return a new QuerySet. So you can't get a `QuerySet` of several objects, update an attribute on one object, save it, and see the result in the same `QuerySet`.

There's an `update()` method on `QuerySet`, but it doesn't call custom `save()` methods or send `pre_save` or `post_save` signals.

The `defer()` and `only()` methods allow you get only the specified fields, but you get actual `Model` instances. If you access other fields, then a new query is performed. 

The `values()` and `values_list()` get you field values, not `Model` instances. With `flat=True`, `values_list()` gets you a flat list of all values in the `QuerySet`.

The `select_related()` method solves the N+1 problem for `ForeignKey` and `OneToOneField` relations - just one SQL query, instead of one for the first model, and one for each of the N results.

The `prefetch_related()` method solves it for `ManyToManyField`s and `GenericRelation`s - one query for model, relations resolved in python code.

### `Manager`
The high-level interaface. The `Model` that the `Manager` is attached to is available in the `Manager` as `self.model`.

If a model has multiple `Manager`s, the first one defined is the default (`Model._default_manager`).

Should always have a `Manager` on your model that can get alllll the objects.

### `Model`
Representation of the data and the associated logic.
- one class = one table
- one field = one column
- one instance = one row

Uses python metaclasses, via `django.db.models.base.ModelBase`. This is how the `Meta` class property is created. It also creates all those extra methods and properties on the class, like `Model.DoesNotExist`.

`ModelBase` calls `contribute_to_class()` on each thing in the attribute dictionary of the new class. For example, `DateTimeField` has a `contribute_to_class()` method that adds the `get_next_by_<field-name>` method.

### `Field`
Custom fields can pretend to be another field by setting internal type to something that exists already.

`to_python()` converts from DB-level type to correct Python type.

`value_to_string()` converts to string for serialization purposes.

Multiple other methods for preparing values for various kinds of DB-level usage - querying, storage, etc.

### Model Inheritance
Multiple types available:
- abstract parents
 - Indicated by `abstract = True` in model's `Meta` declaration
 - doesn't create a database table
 - subclasses, if not abstract, generate a table with both their own fields and those of the abstract parent
- multi-table
 - no special syntax; just subclass the parent model
- proxy models
 - set `proxy = True` in `Meta`
 - will reuse parent's table and fields; only allowed changes are at the python level
 - can define additional methods, manager, etc.
 - queries return instances of the model queried
 
#### Unmanaged models
Related to proxy models. Set `managed = False` in `Meta`. Wraps a database or view that you don't want Django to... manage. 



## The Forms Library
Major components:
- forms
- fields 
- components
- media support

### Widgets
The low-level Form components:
- one for each type of HTML form control
- handles putting data into form control, and taking it out
- `value_from_datadict()` pulls out that widget's value
- `render()` generates the HTML
- `MultiWidget` is a special class that wraps multiple other widgets
 - useful for things like splitting date/time; single value in DB, often want multiple HTML controls
 
### Fields
- represent data type and validation contrainst
- have widgets associated with them for rendering
- calls `clean()` to validate
 - first calls `to_python()`: convert from what came in on HTTP to the correct python type
 - then calls `validate()`: Field's built-in validation
 - finally, calls `run_validators()`: custom validators added in the Field's `validators` kwarg
 - return python value, or raise `ValidationError`
- Choosing a validation scheme
 - `to_python`: when validation constraint is tightly tied to the data type
 - `validate`: when validation is intrinsic to the field (like email addresses)
 - `validators`: when the basic field does almost all the validation you need and it's simpler than writing a whole new field
- error messages
 - django has a lot of special ones
- every field actually has two widgets: regular and hidden versions

### Forms
- Also uses a metaclass, but probably shouldn't. It just keeps track of the order that fields were added to the class.
- Instantiating a `Form` with data will cause it to wrap its fields with instances with `BoundField`.
- Validation
 - Form's `clean()` method
 - happens after field valiation
 - error messages are instances of `ErrorDict`, `ErrorList`
- Displaying
 - default representation is an HTML table
 - can also output `as_p` or `as_ul`
 - if you want to customize the output, look at the `_html_output()` method
 - search for django forms 508 accessibility for examples/libraries of ensuring Section 508 accessibility compliance
 
 ### ModelForms
 